In [26]:
import pandas as pd
import numpy as np
import os

os.listdir('../input')

['train.csv', 'submission_popular.csv', 'item_metadata.csv', 'test.csv']

In [28]:
sub_ex = pd.read_csv('../input/submission_popular.csv')

In [6]:
test = pd.read_csv('../input/test.csv')
test.head()

,user_id,session_id,timestamp,step,action_type,reference,platform,city,device,current_filters,impressions,prices
0,004A07DM0IDW,1d688ec168932,1541555614,1,interaction item image,2059240,CO,"Santa Marta, Colombia",mobile,NaN,NaN,NaN
1,004A07DM0IDW,1d688ec168932,1541555614,2,interaction item image,2059240,CO,"Santa Marta, Colombia",mobile,NaN,NaN,NaN
2,004A07DM0IDW,1d688ec168932,1541555696,3,clickout item,1050068,CO,"Santa Marta, Colombia",mobile,NaN,2059240|2033381|1724779|127131|399441|103357|1...,70|46|48|76|65|65|106|66|87|43|52|44|60|61|50|...
3,004A07DM0IDW,1d688ec168932,1541555707,4,clickout item,1050068,CO,"Santa Marta, Colombia",mobile,NaN,2059240|2033381|1724779|127131|399441|103357|1...,70|46|48|76|65|65|106|66|87|43|52|44|60|61|50|...
4,004A07DM0IDW,1d688ec168932,1541555717,5,clickout item,1050068,CO,"Santa Marta, Colombia",mobile,NaN,2059240|2033381|1724779|127131|399441|103357|1...,70|46|48|76|65|65|106|66|87|43|52|44|60|61|50|...


In [7]:
test2 = test[(test.action_type != 'clickout item') & (test.action_type != 'search for poi')]

In [8]:
test2.action_type.value_counts()

interaction item image     2578455
filter selection            193499
change of sort order        117826
search for destination      115144
interaction item info        68101
interaction item rating      53704
search for item              43065
interaction item deals       42147
Name: action_type, dtype: int64

In [17]:
interact_ = pd.DataFrame(test2.groupby(['session_id','reference'])['reference'].count())
interact_.columns = ['count']
interact_.reset_index(inplace = True)
interact_.head(10)

,session_id,reference,count
0,000177b850519,17011,1
1,000177b850519,17266,1
2,00017b3b2c136,6698142,9
3,0001d7853e07d,888605,2
4,0001d7853e07d,906385,2
5,0001d7853e07d,"Killarney, Ireland",1
6,000246890bfc6,98916,1
7,000246890bfc6,98934,8
8,000246890bfc6,98938,2
9,000246890bfc6,Best Value,2


In [24]:
#drop not integer
interact_ = interact_[interact_.reference.apply(lambda x : x.isnumeric())]
interact_.head(10)

,session_id,reference,count
0,000177b850519,17011,1
1,000177b850519,17266,1
2,00017b3b2c136,6698142,9
3,0001d7853e07d,888605,2
4,0001d7853e07d,906385,2
6,000246890bfc6,98916,1
7,000246890bfc6,98934,8
8,000246890bfc6,98938,2
15,0003005fe4aa2,2418986,1
16,000305995c580,155800,18


In [37]:
interact_ = interact_.groupby('session_id').apply(lambda x : x.sort_values(['count'], ascending = False)).reset_index(drop = True)
interact_.head(10)

,session_id,reference,count
0,000177b850519,17011,1
1,000177b850519,17266,1
2,00017b3b2c136,6698142,9
3,0001d7853e07d,888605,2
4,0001d7853e07d,906385,2
5,000246890bfc6,98934,8
6,000246890bfc6,98938,2
7,000246890bfc6,98916,1
8,0003005fe4aa2,2418986,1
9,000305995c580,155800,18


# prepareing submissions

In [47]:
def get_submission_target(df) :
    #reference = NA, action type clickout item인 경우
    mask = (df.reference.isnull()) & (df.action_type == 'clickout item')
    df_out = df[mask]
    
    return df_out

df_target = get_submission_target(test)
df_target.head()

,user_id,session_id,timestamp,step,action_type,reference,platform,city,device,current_filters,impressions,prices
6,004A07DM0IDW,1d688ec168932,1541555799,7,clickout item,NaN,CO,"Santa Marta, Colombia",mobile,NaN,2059240|2033381|1724779|127131|399441|103357|1...,70|46|48|76|65|65|106|66|87|43|52|44|60|61|50|...
8,009RGHI3G9A3,f05ab0de907e2,1541570940,2,clickout item,NaN,IN,"Nathdwara, India",mobile,NaN,10884872|7065316,64|28
10,00Y1Z24X8084,26b6d294d66e7,1541651823,2,clickout item,NaN,PH,"Iloilo City, Philippines",mobile,NaN,2714480|4476010|3843244|3833012|9017890|198100...,74|14|22|38|55|44|28|34|23|27|12|108|19|21|36|...
15,01V3WDTDM5CU,07628a0f5be0b,1541575643,5,clickout item,NaN,PL,"Wisla, Poland",mobile,Sort by Price,3565720|2947584|4115018|2039671|3836538|801409...,16|18|20|21|22|22|28|28|28|30|30|33|33|35|35|3...
61,02AOAVF9PVYH,4a01c3afbc224,1541681278,46,clickout item,NaN,JP,"Yokohama, Japan",desktop,Hotel|Resort|Sort by Price,1451247|559056|1045096|1963879|693596|1967173|...,80|81|81|82|82|82|83|83|83|85|85|88|88|91|92|9...


In [48]:
def string_to_array(s) :
    
    if isinstance(s, str) :
        out = s.split('|')
    elif math.isnan(s) :
        out = []
    else :
        raise ValueError('Value must be either string of nan')
        
    return out

def explode(df_in, col_expl) :
    #다수의 impressions를 한 행당 한개의 값만 가지도록
    df = df_in.copy()
    df.loc[:, col_expl] = df[col_expl].apply(string_to_array)
    
    df_out = pd.DataFrame({
        col : np.repeat(df[col].values, df[col_expl].str.len())
                       for col in df.columns.drop(col_expl)
    })
    
    df_out.loc[:, col_expl] = np.concatenate(df[col_expl].values)
    df_out.loc[:, col_expl] = df_out[col_expl].apply(int)
    
    return df_out

df_expl = explode(df_target, 'impressions')
df_expl.head()

,action_type,city,current_filters,device,platform,prices,reference,session_id,step,timestamp,user_id,impressions
0,clickout item,"Santa Marta, Colombia",NaN,mobile,CO,70|46|48|76|65|65|106|66|87|43|52|44|60|61|50|...,NaN,1d688ec168932,7,1541555799,004A07DM0IDW,2059240
1,clickout item,"Santa Marta, Colombia",NaN,mobile,CO,70|46|48|76|65|65|106|66|87|43|52|44|60|61|50|...,NaN,1d688ec168932,7,1541555799,004A07DM0IDW,2033381
2,clickout item,"Santa Marta, Colombia",NaN,mobile,CO,70|46|48|76|65|65|106|66|87|43|52|44|60|61|50|...,NaN,1d688ec168932,7,1541555799,004A07DM0IDW,1724779
3,clickout item,"Santa Marta, Colombia",NaN,mobile,CO,70|46|48|76|65|65|106|66|87|43|52|44|60|61|50|...,NaN,1d688ec168932,7,1541555799,004A07DM0IDW,127131
4,clickout item,"Santa Marta, Colombia",NaN,mobile,CO,70|46|48|76|65|65|106|66|87|43|52|44|60|61|50|...,NaN,1d688ec168932,7,1541555799,004A07DM0IDW,399441


In [51]:
df_expl[['user_id', 'session_id', 'timestamp', 'step','impressions']].head()

,user_id,session_id,timestamp,step,impressions
0,004A07DM0IDW,1d688ec168932,1541555799,7,2059240
1,004A07DM0IDW,1d688ec168932,1541555799,7,2033381
2,004A07DM0IDW,1d688ec168932,1541555799,7,1724779
3,004A07DM0IDW,1d688ec168932,1541555799,7,127131
4,004A07DM0IDW,1d688ec168932,1541555799,7,399441


In [55]:
df_expl[['user_id', 'session_id', 'timestamp', 'step','impressions']].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5763399 entries, 0 to 5763398
Data columns (total 5 columns):
user_id        object
session_id     object
timestamp      int64
step           int64
impressions    int64
dtypes: int64(3), object(2)
memory usage: 219.9+ MB


In [56]:
interact_.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 300615 entries, 0 to 300614
Data columns (total 3 columns):
session_id    300615 non-null object
reference     300615 non-null object
count         300615 non-null int64
dtypes: int64(1), object(2)
memory usage: 6.9+ MB


In [58]:
#interact_.reference -> numeric
interact_['reference'] = pd.to_numeric(interact_['reference'])

In [59]:
interact_.head()

,session_id,reference,count
0,000177b850519,17011,1
1,000177b850519,17266,1
2,00017b3b2c136,6698142,9
3,0001d7853e07d,888605,2
4,0001d7853e07d,906385,2


In [61]:
df_expl = df_expl[['user_id', 'session_id', 'timestamp', 'step','impressions']].merge(interact_, left_on = ['session_id','impressions'],
                                                                           right_on = ['session_id','reference'], how = 'left')
df_expl.head()

,user_id,session_id,timestamp,step,impressions,reference,count
0,004A07DM0IDW,1d688ec168932,1541555799,7,2059240,2059240.0,2.0
1,004A07DM0IDW,1d688ec168932,1541555799,7,2033381,NaN,NaN
2,004A07DM0IDW,1d688ec168932,1541555799,7,1724779,NaN,NaN
3,004A07DM0IDW,1d688ec168932,1541555799,7,127131,NaN,NaN
4,004A07DM0IDW,1d688ec168932,1541555799,7,399441,NaN,NaN


In [62]:
df_expl['impressions'] = df_expl['impressions'].astype(str)

In [64]:
df_expl.sort_values(['user_id', 'session_id', 'timestamp', 'step','count'],
                   ascending = [True,True,True,True,False])
df_expl.head()

,user_id,session_id,timestamp,step,impressions,reference,count
0,004A07DM0IDW,1d688ec168932,1541555799,7,2059240,2059240.0,2.0
1,004A07DM0IDW,1d688ec168932,1541555799,7,2033381,NaN,NaN
2,004A07DM0IDW,1d688ec168932,1541555799,7,1724779,NaN,NaN
3,004A07DM0IDW,1d688ec168932,1541555799,7,127131,NaN,NaN
4,004A07DM0IDW,1d688ec168932,1541555799,7,399441,NaN,NaN


In [66]:
mysub = df_expl.groupby(['user_id', 'session_id', 'timestamp', 'step'])['impressions'].apply(lambda x : ''.join(x)).to_frame().reset_index()

In [67]:
mysub.rename(columns = {'impressions' : 'item_recommendations'}, inplace = True)
mysub.head()

,user_id,session_id,timestamp,step,item_recommendations
0,000324D9BBUC,89643988fdbfb,1541593942,10,9234071729121105079297171353141106315218213090...
1,0004Q49X39PY,9de47d9a66494,1541641157,1,2213014318484210213134450424244863723812004350...
2,0004Q49X39PY,beea5c27030cb,1541561202,1,3812004350515032028942292254198422922278962227...
3,00071784XQ6B,9617600e1ba7c,1541630328,2,2272130675591612122727228542281922764147818922...
4,0008BO33KUQ0,2d0e2102ee0dc,1541636411,6,5078612176280166958750206613525304342348645183...


In [68]:
mysub.to_csv('../sub/190505.csv', index  = False)